In [2]:
# reload local packages automatically
%load_ext autoreload
%autoreload 2

# import NumPy and the PyTeNet package
import numpy as np
import pytenet as ptn
from pprint import pprint
import qutip as qt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# <center> Here start tests from 24 DEC

In [92]:
from opentn.channels import quantum_channel
from opentn.tensors import MPS, MPO, quantum_mpo_mps
from opentn.circuits import quantum_circuit, partial_trace
from opentn.entanglement import partial_transpose_two, determine_entanglement

In [59]:
# defining the |0> and |1> for facility
# define a pure state. note: need to define complex for latter use of += operation.
up = np.array([[1,0]], dtype=np.complex128).T
down = np.array([[0,1]], dtype=np.complex128).T

# same for |+> and |-> states
plus = (up + down)/np.sqrt(2)
minus = (up - down)/np.sqrt(2)

# <center> Testing the statevector resulting from MPS of bell state

## <center> 1. $ \ket{\phi^-} = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)$
Hint: to create the MPS, just imagine all the possible combinations 00, 01, 10, 11 and the inner product they would correspond to

In [93]:
A0_up = [[1/np.sqrt(2), 0]]
A0_down = [[0, -1/np.sqrt(2)]]
A0 = np.array([A0_up,A0_down]) # shape: (2,1,2)
A0 = np.transpose(A0, [1,0,2]) # shape: (1,2,2)

A1_up = [[0],[1]]
A1_down = [[1],[0]]
A1 = np.array([A1_up,A1_down]) # shape: (2,2,1)
A1 = np.transpose(A1,[1,0,2]) # shape: (2,2,1)

phi_min = MPS([A0,A1])
phi_min.merge_mps_tensor_pair() # (1, 4, 1)
phi_min


(array([[[ 0.        ],
        [ 0.70710678],
        [-0.70710678],
        [ 0.        ]]]))

In [94]:
# checking normalization of these tensors
np.tensordot(A1,A1.conj(), axes=((2,1), (2,1))) # (vL) (n) vR , (vL*) (n*) vR* -> vR vR*

array([[1, 0],
       [0, 1]])

## <center> 2. $ \ket{\psi^+} = \frac{1}{\sqrt{2}}(\ket{00} + \ket{11})$

In [95]:
B0_up = [[1/np.sqrt(2), 0]]
B0_down = [[0, 1/np.sqrt(2)]]
B0 = np.array([B0_up, B0_down]) # shape: (2,1,2)
B0 = np.transpose(B0, [1,0,2]) # shape: (1,2,2)

B1_up = [[1],[0]]
B1_down = [[0],[1]]
B1 = np.array([B1_up,B1_down]) # shape: (2,2,1)
B1 = np.transpose(B1,[1,0,2]) # shape: (2,2,1)

phi_plus = MPS([B0,B1])
phi_plus.merge_mps_tensor_pair()
phi_plus

(array([[[0.70710678],
        [0.        ],
        [0.        ],
        [0.70710678]]]))

# <center> Simulation of quantum channel for our system

## $$\rho^{out} = \mathcal{E}(\rho) = \sum_k = E_k \rho E^\dagger_k$$

In [7]:
#define the krauss operators for the amplitude damping channel
gamma = 0.5
E0 = np.array([[1,0],[0,np.sqrt(1-gamma)]], dtype=np.complex128)
E1 = np.array([[0,np.sqrt(gamma)],[0,0]], dtype=np.complex128)
krauss_list = [E0, E1]

In [8]:
#calculate the output of the quantum channel as defined by krauss_list: |0>
quantum_channel(up,krauss_list)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [9]:
#calculate the output of the quantum channel as defined by krauss_list: |+>
quantum_channel(plus,krauss_list)

array([[0.75      +0.j, 0.35355339+0.j],
       [0.35355339+0.j, 0.25      +0.j]])

In [10]:
#calculate the output of the quantum channel as defined by krauss_list: |->
quantum_channel(down,krauss_list)

array([[0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])

In [11]:
# maximally mixed state:
rho_mixed = (np.outer(up,up) + np.outer(down,down))/2
quantum_channel(rho_mixed,krauss_list)

array([[0.75+0.j, 0.  +0.j],
       [0.  +0.j, 0.25+0.j]])

# Simulation of quantum circuit for physical system = $q_0$ and environment = $q_1$: 
## $$CR_y(\theta) \rightarrow CNOT_{1->0} = U_{PE}$$
## $$\rho_{PE}^{out} = U_{PE} \rho_{PE} U_{PE}^{\dagger} $$

In [12]:
# creating the unitary operator that acts both on the system and environment
gamma = 0.5
U_PE = np.array([
    [1,0,0,0],
    [0,0, np.sqrt(gamma), np.sqrt(1-gamma)],
    [0,0, np.sqrt(1-gamma), -np.sqrt(gamma)],
    [0,1,0,0]
                ], dtype=np.complex128)

In [13]:
quantum_circuit([up, up], U_PE) # equivalent to physical in pure state |1> & environment initialized in |0> (see Nielsen and Chuang)

(array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]),
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]))

In [14]:
quantum_circuit([plus, up], U_PE) # equivalent to physical in pure state |1> & environment initialized in |0> (see Nielsen and Chuang)

(array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]),
 array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]))

In [15]:

quantum_circuit([down, up], U_PE) # equivalent to physical in pure state |1> & environment initialized in |0> (see Nielsen and Chuang)

(array([[0.5+0.j, 0. +0.j],
        [0. +0.j, 0.5+0.j]]),
 array([[0.5+0.j, 0. +0.j],
        [0. +0.j, 0.5+0.j]]))

In [16]:
rho_mixed = (np.outer(up,up) + np.outer(down,down))/2
quantum_circuit([rho_mixed, up], U_PE) # equivalent to physical in mixed state & environment initialized in |0> (see Nielsen and Chuang)

(array([[0.75+0.j, 0.  +0.j],
        [0.  +0.j, 0.25+0.j]]),
 array([[0.75+0.j, 0.  +0.j],
        [0.  +0.j, 0.25+0.j]]))

## <center> Tests for Initially entangled states

In [17]:
rho_entangled = np.outer(phi_plus[0,:,0],phi_plus[0,:,0])
# Using U as the "circuit" over the initial rho_PE of the full system
rho_out = U_PE@rho_entangled@U_PE.conj().T

partial_trace(rho_out, 2, 2)    

(array([[ 0.75      +0.j, -0.35355339+0.j],
        [-0.35355339+0.j,  0.25      +0.j]]),
 array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]))

In [18]:
rho_entangled = np.outer(phi_min[0,:,0],phi_min[0,:,0])
# Using U as the "circuit" over the initial rho_PE of the full system
rho_out = U_PE@rho_entangled@U_PE.conj().T

partial_trace(rho_out, 2, 2)    

(array([[ 0.25      +0.j, -0.35355339+0.j],
        [-0.35355339+0.j,  0.75      +0.j]]),
 array([[ 0.25      +0.j, -0.35355339+0.j],
        [-0.35355339+0.j,  0.75      +0.j]]))

In [19]:
partial_trace(rho_entangled,2,2)

(array([[0.5, 0. ],
        [0. , 0.5]]),
 array([[0.5, 0. ],
        [0. , 0.5]]))

In [20]:
I4 = np.kron(np.eye(2)/2, np.eye(2)/2)
partial_trace(I4, 2, 2)

(array([[0.5, 0. ],
        [0. , 0.5]]),
 array([[0.5, 0. ],
        [0. , 0.5]]))

## <center> Checking entanglement of mixed states

In [21]:
# testing qutip functions to make sure I understand them
from qutip.states import (state_index_number, state_number_index,
                          state_number_enumerate)
                          
for state in state_number_enumerate([2,2]):
    print(state)

print(state_number_index([2,2], np.choose([0,1],[(0,1),(1,0)])))
print(state_number_index([2,2], np.choose([0,1],[(1,0),(0,1)])))

phi_p = qt.bell_state()
print(phi_p.dims)
rho_p = qt.ket2dm(phi_p)
rho_p.dims # len rho_p.dims[0] ==  how many qubits (systems) are involved

(0, 0)
(0, 1)
(1, 0)
(1, 1)
0
3
[[2, 2], [1, 1]]


[[2, 2], [2, 2]]

In [22]:
rho_entangled #dims: 2, 2

array([[ 0. ,  0. , -0. ,  0. ],
       [ 0. ,  0.5, -0.5,  0. ],
       [-0. , -0.5,  0.5, -0. ],
       [ 0. ,  0. , -0. ,  0. ]])

In [23]:
partial_transpose_two(rho_entangled)

array([[ 0. +0.j,  0. +0.j, -0. +0.j, -0.5+0.j],
       [ 0. +0.j,  0.5+0.j,  0. +0.j,  0. +0.j],
       [-0. +0.j,  0. +0.j,  0.5+0.j, -0. +0.j],
       [-0.5+0.j,  0. +0.j, -0. +0.j,  0. +0.j]])

In [24]:
determine_entanglement(rho_entangled)

True

In [25]:
I4.shape

(4, 4)

In [26]:
determine_entanglement(I4)

False

## Note:
I misunderstood and I was assuming rho_ent =  rho_sys x rho_env
which makes no sense since it is literally the definition of product state. 

## Note2:
One thing I can do to check if a state is entangled or not is using the partial transposition criteria (see Cirac Notes)
-> idea is that if the partial transpose (not partial trace) has any negative values, then the initial state was entangled

Taking the partial trace and checking if it is maximally mixed state is not always a guarantee since we could have simply done I/2 x I/2 and
partial traces would still be maximally mixed even tho the state is not entangled

## List of entangled for sure:
- $\rho = \ket{\psi_{i}}\bra{\psi_{i}}$ for any $i \in$ bell states
- $\rho(f) = f\ket{\psi}\bra{\psi} + (1-f)\ket{\phi}\bra{\phi}$ for $f \neq 0.5$

## Problem:
I want to find something that would be the right result if I try to do the simulation on krauss representation for physical system. Do i just take the partial trace?
but for entangled states this would be maximally mixed state for all, so:

can i recover original density matrix from partial trace??

Idea: look at original definition of sitelispring dilation



# <center> MPO-MPS

In [27]:
"""
   This is what convetion will like for MPS and MPO from now on

        _____      _____ 
       /     \    /     \
    ---|0 A 2|--- |0 B 2|--- 
       \__1__/    \__1__/
          |          |
     
        __|__      __|__
       /  2  \    /  2  \
    ---|0 W 1|--- |0 V 1|---
       \__3__/    \__3__/
          |          |

I do this convetion so it agrees with what I implemented in the CMMP project

MPO: D[i], D[i+1], n[i], m[i] = (0,1,2,3)
virtual_out, virtual_in, physical_in, physical_out == left, right, up, down
where out == row and in == column
"""


'\n   This is what convetion will like for MPS and MPO from now on\n\n        _____      _____ \n       /     \\    /         ---|0 A 2|--- |0 B 2|--- \n       \\__1__/    \\__1__/\n          |          |\n     \n        __|__      __|__\n       /  2  \\    /  2      ---|0 W 1|--- |0 V 1|---\n       \\__3__/    \\__3__/\n          |          |\n\nI do this convetion so it agrees with what I implemented in the CMMP project\n\nMPO: D[i], D[i+1], n[i], m[i] = (0,1,2,3)\nvirtual_out, virtual_in, physical_in, physical_out == left, right, up, down\nwhere out == row and in == column\n'

# <center> Circuit as MPO


In [102]:
# the states in our state machine are: len(a,b,c,d,e,f) = 6

gamma = 0.5
identity = np.eye(2)
zero = np.zeros((2,2))

ry = np.array([
    [np.sqrt(1-gamma), -np.sqrt(gamma)],
    [np.sqrt(gamma), np.sqrt(1-gamma)]
                ], dtype=np.complex128)

W0 = np.array([
            [zero, np.outer(up,up), np.outer(down,down), np.outer(down,up), np.outer(up,down), zero]
                ])

W1 = np.array([
            [zero],
            [np.outer(up,up)],
            [np.outer(up,up)@ry],
            [np.outer(down,down)],
            [np.outer(down,down)@ry],
            [zero]
                ])

# Amplitude damping MPO acting on two qubits
Wlist = [W0, W1]
Wlist = [W.transpose((0,1,3,2)) for W in Wlist] # vL vR i j

In [112]:
mpo = MPO(Wlist)
mps_uu = MPS.generate_product_MPS([up, up]) #each has vL i vR
quantum_mpo_mps(mps_uu, mpo)

(array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]),
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]))

In [111]:
mpo = MPO(Wlist)
mps_pu = MPS.generate_product_MPS([plus, up]) #each has vL i vR
quantum_mpo_mps(mps_pu, mpo)

(array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]),
 array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]))

In [110]:
mpo = MPO(Wlist)
mps_du= MPS.generate_product_MPS([down, up]) #each has vL i vR
quantum_mpo_mps(mps_du, mpo)

(array([[0.5+0.j, 0. +0.j],
        [0. +0.j, 0.5+0.j]]),
 array([[0.5+0.j, 0. +0.j],
        [0. +0.j, 0.5+0.j]]))

# Testing what happens if we change the environment initial state

In [109]:
mpo = MPO(Wlist)
mps_pp = MPS.generate_product_MPS([plus, plus]) #each has vL i vR
quantum_mpo_mps(mps_pp , mpo)

(array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]),
 array([[0.25      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.75      +0.j]]))

In [115]:
mpo = MPO(Wlist)
mps_pd = MPS.generate_product_MPS([plus, down]) #each has vL i vR
quantum_mpo_mps(mps_pd, mpo)

(array([[0.25      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.75      +0.j]]),
 array([[ 0.25      +0.j, -0.35355339+0.j],
        [-0.35355339+0.j,  0.75      +0.j]]))

In [116]:
mpo = MPO(Wlist)
mps_dm = MPS.generate_product_MPS([down, minus]) #each has vL i vR
quantum_mpo_mps(mps_dm, mpo)

(array([[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]),
 array([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]))

# try for a bell state:

In [119]:
mpo = MPO(Wlist)
psi_plus = MPS([B0, B1])
quantum_mpo_mps(psi_plus, mpo)

(array([[ 0.75      +0.j, -0.35355339+0.j],
        [-0.35355339+0.j,  0.25      +0.j]]),
 array([[0.75      +0.j, 0.35355339+0.j],
        [0.35355339+0.j, 0.25      +0.j]]))